<a href="https://colab.research.google.com/github/todayuriku/AIreview/blob/main/AI%E3%83%AC%E3%83%93%E3%83%A5%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なパッケージのインポート
import pandas as pd
import google.generativeai as genai
from google.colab import drive, userdata
import time
import random
from tqdm import tqdm
from google.api_core import exceptions, retry

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
from datetime import datetime

In [ ]:
# Google Driveのマウントとパッケージのインストール
drive.mount('/content/drive')
!pip install google-generativeai

Mounted at /content/drive


In [ ]:
import google.colab
import google.generativeai as genai

In [ ]:
# 行の表示数を設定（Noneを指定すると全ての行を表示）
pd.set_option('display.max_rows', None)

# 列の表示数を設定（Noneを指定すると全ての列を表示）
pd.set_option('display.max_columns', None)

In [ ]:
# API鍵の設定
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# 1. API Keyの確認
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API Keyが設定されていません")
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"API Keyの設定エラー: {e}")

In [ ]:
# Gemini Proモデルの初期化
# gemini_pro = genai.GenerativeModel("gemini-1.5-flash")
gemini_pro = genai.GenerativeModel("gemini-1.0-pro")

# APIのテスト
prompt = "こんにちは"
response = gemini_pro.generate_content(prompt)
print(response.text)

こんにちは、ご質問は何ですか?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **dfにレビューデータを入れる**

In [ ]:
df = pd.read_csv("")

# **有用・非有用判定**

In [ ]:
@retry.Retry(predicate=retry.if_exception_type(exceptions.ResourceExhausted))
def analyze_review(review_data):
    """レビューの有用性を分析する関数

    Args:
        review_data: レビュータイトルとテキストを含む辞書またはSeries
    Returns:
        str: 分析結果のテキスト
    """
    review_text = f"タイトル: {review_data['review_title']}\n本文: {review_data['review']}"
    prompt = f"""
    以下に入力する商品レビューのと本文が、
    {review_text}
    """
    try:
        response = gemini_pro.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error analyzing review: {e}")
        return "ERROR"

def analyze_reviews_batch(df, start_idx=0, batch_size=10):
    """指定された開始位置からバッチサイズ分のレビューを分析する

    Args:
        df: 分析対象のデータフレーム
        start_idx: 開始インデックス
        batch_size: バッチサイズ
    Returns:
        analyzed_df: 分析結果を含むデータフレーム
        last_idx: 最後に処理したインデックス
    """
    end_idx = min(start_idx + batch_size, len(df))
    batch = df.iloc[start_idx:end_idx].copy()

    results = []
    for _, row in tqdm(batch.iterrows(), total=len(batch), desc=f"Analyzing {start_idx} to {end_idx}"):
        result = analyze_review(row)
        results.append(result)
        time.sleep(3)  # API制限対策

    batch['analysis_result'] = results
    return batch, end_idx

# 進捗を保存しながら分析を実行
def run_analysis_with_checkpoints(input_df, checkpoint_path='review_analysis_checkpoint.csv',
                                batch_size=10, start_from=0):
    """チェックポイントを保存しながらレビュー分析を実行する

    Args:
        input_df: 入力データフレーム
        checkpoint_path: チェックポイントファイルのパス
        batch_size: バッチサイズ
        start_from: 開始インデックス
    """
    all_results = []
    current_idx = start_from

    try:
        while current_idx < len(input_df):
            # バッチ処理の実行
            analyzed_batch, last_idx = analyze_reviews_batch(
                input_df,
                start_idx=current_idx,
                batch_size=batch_size
            )

            # 結果の保存
            all_results.append(analyzed_batch)
            current_results = pd.concat(all_results)
            current_results.to_csv(checkpoint_path)

            print(f"Checkpoint saved at index {last_idx}")
            current_idx = last_idx

            time.sleep(10)  # バッチ間の待機

    except Exception as e:
        print(f"Error occurred at index {current_idx}: {e}")
        print("You can resume from this index later")

    finally:
        # 最終結果の結合
        final_results = pd.concat(all_results) if all_results else pd.DataFrame()
        return final_results

In [ ]:
results_df = run_analysis_with_checkpoints(
    df,
    checkpoint_path='review_analysis_checkpoint.csv',
    batch_size=10
)

In [ ]:
results_df.head()

In [ ]:
results_df["analysis_result"].value_counts()